# RAB Dashboarding Notebook

In [1]:
from ruamel.yaml import YAML
import pandas as pd
import altair as alt
import rab
from zoho import Zoho, CrmRequest

In [4]:
# Load the config settings
yaml=YAML(typ='safe')
with open("config.yml", "r") as config_file:
    config = yaml.load(config_file)

zoho_config = config['zoho']

In [5]:
zoho_client = Zoho(
    org_id = zoho_config['org_id'], 
    client_id = zoho_config['client_id'], 
    client_secret = zoho_config['client_secret'], 
    redirect = zoho_config['redirect'], 
    scope = zoho_config['scope']
)

127.0.0.1 - - [23/Feb/2021 12:46:10] "GET /?code=1000.fc467f257c650bbcbed9c69852d941e0.21a6ebbc41cf695a2c392cca4be0d0c5&location=us&accounts-server=https%3A%2F%2Faccounts.zoho.com& HTTP/1.1" 200 -


In [6]:
request = CrmRequest(zoho_client, 'Invoices')
fields = ['Aircraft_SN', 'Account_Name', 'Engine_Model', 'Engine_Serial_Number', 'Export_Country', 'Product_Details', 'Grand_Total']
request.params.fields = fields
agsur_sales = request.execute()


Added 200 records for a total of 200 records
Added 16 records for a total of 216 records


In [36]:
agsur_sales_df = pd.DataFrame(agsur_sales)
agsur_sales_df[['account_name', 'account_id']]=agsur_sales_df['Account_Name'].apply(lambda x: (x['name'], x['id'])).apply(pd.Series)
agsur_sales_df['model']=agsur_sales_df['Product_Details'].apply(lambda x: x[0]['product']['name'])
agsur_sales_df = agsur_sales_df.drop(["Account_Name", "Product_Details"], axis=1)
agsur_sales_df.head()

,Export_Country,Grand_Total,Engine_Model,Engine_Serial_Number,Aircraft_SN,id,account_name,account_id,model
0,Argentina,1874000.0,PT6A-67AG,PCE-RD0355,802-0856,524414000009979292,Helicópteros del Pacífico Argentina S. A.,524414000009979267,AT-802
1,Paraguay,881595.0,PT6A-15AG,PCE-PD0268,402B-1443,524414000009979227,Aero Chaco S.A.,524414000008575001,AT-402B
2,Brazil,863035.0,PT6A-11AG,PCE-RH0139,402A-1444,524414000009979196,Aero Agrícola Medianeira Ltda,524414000009979171,AT-402A
3,Brazil,1055110.0,PT6A-34AG,PCE-PH1348,504-4037,524414000009979139,Aplimax Aeroagricola Ltda,524414000005114068,AT-504
4,Brazil,1021145.0,PT6A-34AG,PCE-PH1356,502B-3265,524414000009979095,Agripar Aviação Agrícola Ltda,524414000009979060,AT-502B


In [2]:
air_br =  rab.Rab()

Getting the Registro Aeronáutico Brasileiro (RAB) from Brazilian ANAC
File Already Downloaded at data/rab/raw.csv
Cleaning the Raw Registro Aeronáutico Brasileiro (RAB)


In [3]:
air_br.clean.head()

,tail_number,owner_customer_name,owner_other,owner_state,owner_tax_id,operator_customer_name,operator_other,operator_state,operator_tax_id,certifate_num,...,unk_cdmarcanac3,foreign_tail_number,unk_dsgravame,age,owner_tax_id_type,owner_tax_id_print,operator_tax_id_type,operator_tax_id_print,owned_operated,agaircraft
0,PRYYX,AZUL LINHAS AEREAS BRASILEIRAS S.A.,NaN,SP,09296295000160,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,RESERVADAS AS MARCAS,<NA>,CNPJ,09.296.295/0001-60,EMPTY,<NA>,False,False
1,PRZAB,AMT PATRIMONIAL E PARTCIPACOES LTDA,FERNANDO SOARES GURGEL,CE,29327303000103,AMT PATRIMONIAL E PARTCIPACOES LTDA,FERNANDO SOARES GURGEL,CE,29327303000103,121229,...,NaN,NaN,NaN,9,CNPJ,29.327.303/0001-03,CNPJ,29.327.303/0001-03,True,False
2,PRZAD,AIRSHIP DO BRASIL - INDUSTRIA E SERVICOS AEREO...,NaN,SP,07933461000166,AIRSHIP DO BRASIL - INDUSTRIA E SERVICOS AEREO...,NaN,SP,07933461000166,201106,...,NaN,NaN,NaN,1,CNPJ,07.933.461/0001-66,CNPJ,07.933.461/0001-66,True,False
3,PRZAE,ADEMIR BATISTA BRAGA,NaN,SP,84293918868,ADEMIR BATISTA BRAGA,NaN,SP,84293918868,110921,...,NaN,NaN,NaN,10,CPF,842.939.188-68,CPF,842.939.188-68,True,False
4,PRZBB,JOSE CLAUDIO FURLAN,NaN,MG,45158940649,JOSE CLAUDIO FURLAN,NaN,MG,45158940649,111205,...,NaN,NaN,NaN,10,CPF,451.589.406-49,CPF,451.589.406-49,True,False


In [55]:
ag_air = air_br.clean[air_br.clean['agaircraft']].copy()
ag_air = ag_air.merge(agsur_sales_df, left_on='serial', right_on="Aircraft_SN")

In [56]:
# Ag Aircraft Sales by year grouped by Mfg 2010-2021

In [ ]:
# Cumlative Ag Aircraft in Brazil with current IAM and CAM status grouped by Mfg, Model, Engine Type

In [ ]:
# Percent and count of Ag Aircraft Sales / Air Tractor sale by AgSur 2010-2021

In [58]:
# Percent and count of Ag Aircraft Sales / Air Tractor sale by AgSur cumulative grouped by state